In [1]:
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm
from typing import Iterable, Callable
from questionanswering import squad

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
df = squad.parse_json_file("input/train-v2.0.json")
questions = list(df["question"])
contexts = list(df["context"])
df.info()

100%|████████████████████████████████| 442/442 [00:00<00:00, 745.79it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            130319 non-null  object
 1   title         130319 non-null  object
 2   question      130319 non-null  object
 3   answer_text   130319 non-null  object
 4   context       130319 non-null  object
 5   answer_start  130319 non-null  int16 
dtypes: int16(1), object(5)
memory usage: 5.2+ MB


In [4]:
assert len(df) == 130319
assert len(df) == len(set(df["id"]))
df.head()

,id,title,question,answer_text,context,answer_start
0,56be85543aeaaa14008c9063,Beyoncé,When did Beyonce start becoming popular?,in the late 1990s,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",269
1,56be85543aeaaa14008c9065,Beyoncé,What areas did Beyonce compete in when she was growing up?,singing and dancing,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",207
2,56be85543aeaaa14008c9066,Beyoncé,When did Beyonce leave Destiny's Child and become a solo singer?,2003,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",526
3,56bf6b0f3aeaaa14008c9601,Beyoncé,In what city and state did Beyonce grow up?,"Houston, Texas","Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",166
4,56bf6b0f3aeaaa14008c9602,Beyoncé,In which decade did Beyonce become famous?,late 1990s,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Bil

In [5]:
s = pd.Series(np.where(df["answer_start"]==-1, True, False))
assert s.value_counts()[True] == 43498
s.value_counts(normalize=True)

False    0.666219
True     0.333781
dtype: float64

In [6]:
df[df["answer_start"] == -1].sample(5).head(5)

,id,title,question,answer_text,context,answer_start
51266,5a0cea77f5590b0018dab652,Bill_%26_Melinda_Gates_Foundation,What has the United States tried to do to increase the number of college graduates?,,"One of the foundation's goals is to lower poverty by increasing the number of college graduates in the United States, and the organization has funded ""Reimagining Aid Design and Delivery"" grants to think tanks and advocacy organizations to produce white papers on ideas for changing the current system of federal financial aid for college students, with a goal of increasing graduation rates. One of the ways the foundation has sought to increase the number of college graduates is to get them through college faster, but that idea has received some pushback from organizations of universities and colleges.",-1
82021,5acfed0477cf76001a68651c,Tibet,What happened to eastern Kham in 1782?,,"Qing dynasty rule in Tibet began with their 1720 expedition to the country when they expelled the invading Dzungars. Amdo came under Qing control in 1724, and eastern Kham was incorporated into neighbouring Chinese provinces in 1728. Meanwhile, the Qing government sent resident commissioners called Ambans to Lhasa. In 1750 the Ambans and the majority of the Han Chinese and Manchus living in Lhasa were killed in a riot, and Qing troops arrived quickly and suppressed the rebels in the next year. Like the preceding Yuan dynasty, the Manchus of the Qing dynasty exerted military and administrative control of the region, while granting it a degree of political autonomy. The Qing commander publicly executed a number of supporters of the rebels and, as in 1723 and 1728, made changes in the political structure and drew up a formal organization plan. The Qing now restored the Dalai Lama as ruler, leading the governing council called Kashag, but elevated the role of Ambans to include more direct involvement in Tibetan internal affairs. At the same time the Qing took steps to counterbalance the power of the aristocracy by adding officials recruited from the clergy to key posts.",-1
111667,5ad0e4d6645df0001a2d07da,Premier_League,What does the Football League Championship club do with a portion of TV revenue?,,"The Premier League distributes a portion of its television revenue to clubs that are relegated from the league in the form of ""parachute payments"". Starting with the 2013–14 season, these payments are in excess of £60 million over four seasons. Though designed to help teams adjust to the loss of television revenues (the average Premier League team receives £55 million while the average Football League Championship club receives £2 million), critics maintain that the payments actually widen the gap between teams that have reached the Premier League and those that have not, leading to the common occurrence of teams ""bouncing back"" soon after their relegation. For some clubs who have failed to win immediate promotion back to the Premier League, financial problems, including in some cases administration or even liquidation have followed. Further relegations down the footballing ladder have ensued for several clubs unable to cope with the gap.",-1
118097,5acfae8377cf76001a685839,Namibia,Which sub-Saharan African country has a large army?,,"Namibia follows a largely independent foreign policy, with persisting affiliations with states that aided the independence struggle, including Cuba. With a small army and a fragile economy, the Namibian Government's principal foreign policy concern is developing strengthened ties within the Southern African region. A dynamic member of the Southern African Development Community, Namibia is a vocal advocate for greater regional integration. Namibia became the 160th member of the UN on 23 April 1990. On its independence it became the fiftieth member of the Commonwealth of Nations.",-1
64254,5ad02e2877cf76001a686d29,Late_Middle_Ages,Which continents did Vasco da Gama circumnavigate in 1489?,,"After the end of the late Middle 

In [7]:
%%time
col = "qc_length"
df[col] = df["question"].str.len() + df["context"].str.len() 
df[col] = df[col].astype(np.int32)

Wall time: 97 ms


In [8]:
%%time
col = "a_length"
df[col] = df["answer_text"].str.len()
df[col] = df[col].astype(np.int32)

Wall time: 51 ms


In [9]:
def word_length(cols: Iterable) -> Callable:
    def f(row) -> int:
        res = 0
        for col in cols:
            res += len(row[col].split())
        return res
    
    return f

In [10]:
%%time
col = "qc_word_length"
df[col] = df.progress_apply(word_length(["question", "context"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|████████████████████████| 130319/130319 [00:02<00:00, 51982.05it/s]

Wall time: 2.51 s


In [11]:
%%time
col = "a_word_length"
df[col] = df.progress_apply(word_length(["answer_text"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|███████████████████████| 130319/130319 [00:01<00:00, 110531.37it/s]

Wall time: 1.18 s


In [12]:
pretrained_dir = "../pretrained/google/electra-small-discriminator"
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']


In [13]:
pretrained_dir = "../pretrained/albert-base-v2"
sp_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(sp_tokenizer)}\n{sp_tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/albert-base-v2', vocab_size=30000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False)})
['input_ids', 'token_type_ids', 'attention_mask']


In [14]:
pretrained_dir = "../pretrained/distilroberta-base"
bpe_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(bpe_tokenizer)}\n{bpe_tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/distilroberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})
['input_ids', 'attention_mask']


In [15]:
%%time
x = tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_wp_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=130319
Wall time: 58 s


In [16]:
%%time
x = sp_tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_sp_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=130319
Wall time: 1min 41s


In [17]:
%%time
x = bpe_tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_bpe_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'attention_mask'])
len=130319
Wall time: 1min 14s


In [18]:
cols = ["qc_length", "a_length", "qc_word_length", "a_word_length", 
        "qc_wp_length", "qc_sp_length", "qc_bpe_length"]
df[cols].describe(percentiles=percentiles)

,qc_length,a_length,qc_word_length,a_word_length,qc_wp_length,qc_sp_length,qc_bpe_length
count,130319.000000,130319.000000,130319.000000,130319.000000,130319.000000,130319.000000,130319.000000
mean,812.810787,13.423722,129.508138,2.107099,170.726632,174.188384,172.696813
std,308.710471,20.016981,49.604294,3.144031,65.407215,67.240044,65.938716
min,178.000000,0.000000,25.000000,0.000000,35.000000,35.000000,36.000000
1%,254.000000,0.000000,40.180000,0.000000,55.000000,56.000000,57.000000
5%,390.000000,0.000000,62.000000,0.000000,84.000000,86.000000,86.000000
10%,531.000000,0.000000,82.000000,0.000000,108.000000,109.000000,109.000000
20%,596.000000,0.000000,95.000000,0.000000,124.000000,126.000000,125.000000
30%,642.000000,0.000000,102.000000,0.000000,135.000000,137.000000,136.000000
40%,693.000000,4.000000,111.000000,1.000000,146.000000,149.000000,148.000000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              130319 non-null  object
 1   title           130319 non-null  object
 2   question        130319 non-null  object
 3   answer_text     130319 non-null  object
 4   context         130319 non-null  object
 5   answer_start    130319 non-null  int16 
 6   qc_length       130319 non-null  int32 
 7   a_length        130319 non-null  int32 
 8   qc_word_length  130319 non-null  int32 
 9   a_word_length   130319 non-null  int32 
 10  qc_wp_length    130319 non-null  int16 
 11  qc_sp_length    130319 non-null  int16 
 12  qc_bpe_length   130319 non-null  int16 
dtypes: int16(4), int32(4), object(5)
memory usage: 8.0+ MB


In [20]:
%%time
df.to_parquet("output/train.parquet", index=False)

Wall time: 380 ms
